<a href="https://colab.research.google.com/github/etuckerman/surf_NLP/blob/main/KSF_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas scikit-learn numpy sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [2]:
# Step 1: Import necessary libraries
import pandas as pd
import re
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sentence_transformers import SentenceTransformer
import pickle

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Step 2: Load the CSV file
from google.colab import files
uploaded = files.upload()

Saving messages.csv to messages.csv


In [4]:
# Assuming the CSV file is called 'messages.csv'
df = pd.read_csv(list(uploaded.keys())[0])

In [5]:
# Step 3: Data Cleaning
# Convert any non-string values in the 'Message' column to an empty string
df['Message'] = df['Message'].astype(str)

# Apply regex to remove encoded emojis
encoded_emoji_pattern = re.compile(r'[^\x00-\x7F]+')
df['Message'] = df['Message'].apply(lambda x: re.sub(encoded_emoji_pattern, '', x))

# Remove messages with no full words or very short messages
short_responses = ['ok', 'lol', 'haha', 'yes', 'no', 'sure', 'right', 'cool', 'nah', 'yep', 'nope', 'yeah', 'k', 'hm']
df = df[~df['Message'].str.lower().isin(short_responses)]
df = df[df['Message'].str.contains(r'\b\w{3,}\b')]

# Remove empty or very short messages after cleaning
df = df[df['Message'].str.strip().str.len() > 2]

# Reset index after cleaning
df = df.reset_index(drop=True)

In [6]:
# Step 4: Initialize model with GPU support if available
model = SentenceTransformer('paraphrase-MiniLM-L6-v2', device='cuda')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Function for batch processing of embeddings
def batch_encode(model, texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        embeddings.extend(model.encode(batch))
    return embeddings

In [8]:
# Generate embeddings using batch processing
df['Embedding'] = batch_encode(model, df['Message'].tolist())

In [9]:
# Save embeddings to a file after computation (optional)
with open('embeddings.pkl', 'wb') as f:
    pickle.dump(df['Embedding'].tolist(), f)

# Load embeddings from a file (optional, for future use)
# with open('embeddings.pkl', 'rb') as f:
#     df['Embedding'] = pickle.load(f)

In [10]:
# Step 5: Function to find the most relevant answer using index as recency
def find_answer(query, df, recency_bias=0.01):
    query_embedding = model.encode(query)

    # Calculate cosine similarity between the query and all messages
    df['Similarity'] = df['Embedding'].apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])

    # Apply recency bias based on index
    df['Recency_Score'] = np.exp(-recency_bias * (len(df) - df.index))

    # Combine similarity and recency scores
    df['Score'] = df['Similarity'] * df['Recency_Score']

    # Find the message with the highest score
    best_match = df.loc[df['Score'].idxmax()]
    return best_match['Message']

In [11]:
# Example usage
query = "who is the best surfer?"
answer = find_answer(query, df)
print("Best Answer:", answer)

Best Answer: im a big boy t6 surfer
